<a href="https://colab.research.google.com/github/JoaoMouraSS/Projetos/blob/main/Analise_dos_gastos_dos_deputados_estaduais_de_SP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução

O seguinte projeto visa a análise exploratória dos gastos dos parlamentares do estado de São Paulo.

# *Fluxo do Projeto*


1.   Etapa API – Criação de uma API para consumir dados públicos da ALEPS.
2.   Etapa Ingestão – Transferir os dados para  framework Pandas.
3.   Etapa Modelagem – Cruzar os dados de 2 fontes da API e criar uma análise de despesas de gastos.
4.   Etapa Armazenar – Criar um banco de dados e fazer a ingestão dos dados para futuras consultas.
5.   Analisar os Dados – Utilizar frameworks de Data Visualization para entender o que contem nos dados



In [2]:
# Consumir a api
import requests

# Ajustar os dados
import numpy as np
import pandas as pd

# Gerar gráficos
import matplotlib.pyplot as plt
import plotly.express as px

# Ignorar filtros
from warnings import filterwarnings
filterwarnings('ignore')

# *Consumindo a api*

In [3]:
# Links
despesas = 'http://www.al.sp.gov.br/repositorioDados/deputados/despesas_gabinetes.xml'
cadastro = 'https://www.al.sp.gov.br/repositorioDados/deputados/deputados.xml'

# Pegando as informações
dados_despesas = requests.get(despesas)
dados_cadastro = requests.get(cadastro)

# Verificando o retorno da api
print(dados_despesas, dados_cadastro)

<Response [200]> <Response [200]>


# *Transformando para o objeto pd.dataframe e fazendo alterações*

In [5]:
# Convertendo os dados para um dataframe
df_despesas = pd.read_xml(dados_despesas.content)
df_cadastro = pd.read_xml(dados_cadastro.content)

# *Modelando os dados*

In [24]:
# Removendo colunas desnecessárias
df_despesas = df_despesas[['Ano','Matricula','Valor']]
df_cadastro = df_cadastro[['Matricula','Partido','NomeParlamentar']]

filtro = df_despesas.loc[(df_despesas['Ano']>=2019)]

# Agrupar o valor gasto de cada deputado por ano
analise = filtro.groupby(by = ['Ano', 'Matricula']).sum()['Valor'].reset_index()
analise = analise.pivot(index='Matricula',columns='Ano',values='Valor')
analise = analise.dropna()
analise

# Fazer o cruzamento dos dados entre as tabelas
df_analise = pd.merge(analise,df_cadastro,how='inner',on='Matricula')
df_analise

,Matricula,2019,2020,2021,2022,Partido,NomeParlamentar
0,300098,253434.48,291386.79,322863.63,231948.74,PSDB,Mauro Bragato
1,300188,237587.17,247384.14,300364.14,222276.53,PSDB,Barros Munhoz
2,300205,102936.63,126638.83,84064.04,59692.71,PL,Conte Lopes
3,300217,302073.11,303710.00,370183.68,243565.81,AVANTE,Campos Machado
4,300260,139872.47,125206.46,144298.34,92577.75,AVANTE,Roque Barbiere
...,...,...,...,...,...,...,...
82,300646,154261.19,236119.84,333694.06,275150.47,PL,Tenente Coimbra
83,300647,173487.95,167221.22,210208.62,152804.17,REPUBLICANOS,Tenente Nascimento
84,300648,220277.05,234087.89,269953.47,247932.64,PL,Thiago Auricchio
85,300649,103766.46,210226.82,287744.27,180627.52,PL,Valeria Bolsonaro
